In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

In [4]:
! dir

 Volume in drive C has no label.
 Volume Serial Number is 6C06-A0C3

 Directory of C:\Users\Sridevi\Desktop\sqlalchemy-challenge\solved

11/15/2022  05:48 PM    <DIR>          .
11/15/2022  05:48 PM    <DIR>          ..
11/15/2022  10:08 AM    <DIR>          .ipynb_checkpoints
11/15/2022  05:48 PM           131,826 climate_starter.ipynb
               1 File(s)        131,826 bytes
               3 Dir(s)  158,243,942,400 bytes free


# Reflect Tables into SQLAlchemy ORM

In [5]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func , inspect
from sqlalchemy import desc

In [6]:
# create engine to hawaii.sqlite

engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [7]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine , reflect = True)

In [8]:
# View all of the classes that automap found
#dir(Base.classes)
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
Station = Base.classes.station
Measurement = Base.classes.measurement

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [11]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)


In [12]:
# Collect the names of tables within the database
inspector.get_table_names()

['measurement', 'station']

In [13]:
# Using the inspector to print the column names within the 'Measurement' table and its types
columns = inspector.get_columns('Measurement')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [14]:
# Using the inspector to print the column names within the 'Station' table and its types
columns = inspector.get_columns('Station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


# Exploratory Precipitation Analysis

In [15]:
# # Find the most recent date in the data set.
# for row in session.query(Measurement.date).order_by(desc(Measurement.date)).limit(1).all():
#     print(row)

In [16]:
# Find the most recent date in the data set.
from datetime import datetime
from dateutil.relativedelta import relativedelta    
recent_date = session.query(func.max(Measurement.date)).scalar()


recent_date_minus_year = (datetime.strptime(recent_date, '%Y-%m-%d') - relativedelta(years=1)).strftime('%Y-%m-%d')
print(recent_date_minus_year)
print(recent_date)

2016-08-23
2017-08-23


In [42]:
import pandas as pd
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.
#qry = session.query(Measurement).filter(Measurement.date.between('2016-08-23', '2017-08-23')).limit(10).all()
qry = session.query(Measurement).filter(Measurement.date.between(recent_date_minus_year,recent_date)).all()
#qry = session.query(Measurement).filter(Measurement.date.between('2016-08-23', '2017-08-23')).all()
type(qry)

# #query to dictinory to dataframe
# date_list = []
# prcp_list = []
# for each in qry:
#     date_list.append(each.date)
#     prcp_list.append(each.prcp)
    
# measurement_dict = {'date':date_list, 'prcp':prcp_list }
# pd.DataFrame(measurement_dict, columns=['date','prcp'])




    
    #print(type(each))
   # print(each.id,each.station, each.date ,each.prcp,each.tobs)




# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




,date,prcp
0,2016-08-23,0.00
1,2016-08-24,0.08
2,2016-08-25,0.08
3,2016-08-26,0.00
4,2016-08-27,0.00
...,...,...
2225,2017-08-19,0.09
2226,2017-08-20,NaN
2227,2017-08-21,0.56
2228,2017-08-22,0.50


In [34]:
from sqlalchemy.orm import sessionmaker
import pandas
Session = sessionmaker(bind = engine)
session = Session()
 
#SQLAlCHEMY ORM QUERY TO FETCH ALL RECORDS
df = pandas.read_sql_query(
     #sql = session.query(Measurement).statement,
     sql = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date.between(recent_date_minus_year,recent_date)).statement,
     con = engine
 )

df.set_index('date', inplace=True)
 
print("Type:", type(df))
print()
print(df)



#df = pd.DataFrame(qry, columns=['id','station','date', 'prcp','tobs' ])
# df.set_index('date', inplace=True)





Type: <class 'pandas.core.frame.DataFrame'>

            prcp
date            
2016-08-23  0.00
2016-08-24  0.08
2016-08-25  0.08
2016-08-26  0.00
2016-08-27  0.00
...          ...
2017-08-19  0.09
2017-08-20   NaN
2017-08-21  0.56
2017-08-22  0.50
2017-08-23  0.45

[2230 rows x 1 columns]


In [95]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [13]:
# Design a query to calculate the total number stations in the dataset


In [14]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [15]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [16]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [17]:
# Close Session
#session.close()